# Project 1: Assembling Genomes


   <div class="alert alert-block alert-danger">
    <center>Due: <b>Tuesday, 8 February, 4:59pm</b>.</center> 
   </div>
   
   <div class="alert alert-block alert-warning">
   <center>
       <b>Collaboration and Resource Policy</b>
    </center>
    For this assignment, you are encouraged to work with one other person. Your team must satisfy the constraints mentioned in <a href="https://www.dropbox.com/s/g6z2xncwc4jsplp/csbio-class2-inked.pdf?dl=0">Class 2</a>.
    
   1. You went to different high schools.
   2. You and your partner have different answers to <em>at least one</em> of these questions:
       - What country were you born in?
       - Have you taken any biology courses at UVA?
       - Have you taken cs3102?
       - Have you taken cs4102?
    
We expect most students will have the best learning experience on this assignment by working with a partner, but if you prefer to work alone it is okay to do this assignment on your own.
    
You are permitted (actually _encouraged_) to discuss these problems with anyone you want, including other students in the class. If you do discuss the specific questions in the assignment with anyone other than your assignment partner and the course staff, though, you should list them in the _External resources used_ section below.
    
You are welcome to use any resources you want for this assignment, other than ones that would defeat the purpose of the assignment. This means you should not look at answers or code from any other students in the class (other than your collaboration with your partner), and if you find code that implements the problem you are being asked to do for the assignment, you should not use that code. You should document all external resource you use that are not part of the course materials in the _External resources used_ section below.

**Team submitting this assignment:**  
<div class="alert alert-success">
    <b><em>list each member of your team here, including both your name and UVA computing id
        Colin Crowe (cbc6ghd) </em></b>
</div>

**External resources used:** 
<div class="alert alert-success">
<em>It is not necessary to list the course materials, but if you used any other resources, including discussing problems with students not on your team, list them here.</em>
    
    https://mathworld.wolfram.com/HamiltonianPath.html#:~:text=A%20Hamiltonian%20path%2C%20also%20called,cycle%20(or%20Hamiltonian%20cycle).
    
</div>

In this project, we will explore genome assembly — the process of determining the order of nucleotides from fragmented reads that are produced by sequencing machines. 

Genome assembly maps to a very well defined computer science problem, but can get quite complicated, as problems such as full sequence coverage, finding a good length for reads (the $k$ in $k$-mer), and sequencing errors present challenges for sequencing analysis and accuracy. For the required problems (ones everyone is expected to solve), you will be able to assume perfect coverage and no read errors; for the "challenge" problem (that is considered a bonus, and not something we expect everyone to be able to solve, and perhaps may not even be feasible for anyone to solve) you will not be able to rely on such assumptions.

 <div class="alert alert-block alert-warning">
    
<b>Submission</b>: You should work on this assignment by forking the provided Project 1 repository (you probably already did this, following the instructions posted at [https://computingbiology.github.io/project1](https://computingbiology.github.io/project1). Add the other teammate to as a collaborator with write access (also under the Settings tab) to this repository.
    
You should answer the questions and write your code in this Jupyter Notebook. (We don't expect you to need to use any external files or organize your code outside of the notebook, but if you do, make sure to put everything needed to run your code in your repository.) Parts where you are expected to provide and answer (which could be text that can be written in markdown format in the notebook or Python code that runs in the notebook) are marked in green.
        
When you are ready to submit the assignment, you should create a release of the version you are submitting ([github's directions for how to create a release](https://docs.github.com/en/repositories/releasing-projects-on-github/managing-releases-in-a-repository)) tagged as `submit`. After you've done this, send a message in slack to a channel that includes both team members (so the one sending this message should include the other team member) and all of the course staff (`dave`, `Hyun Jae Cho`, and `Anshuman Suri`) and a link to your repository (feel free to add any other useful comments if you want, but the link to your submission repository is sufficient).

## Getting Started

1. Install basic required packages, should be run only once. You may need to restart the jupyter python kernel (under the Kernel menu) after this. (You can execute this directly in the notebook but running the command below.)

In [2]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


2. Make sure you have [graphviz](https://graphviz.org/) installed on your system. (On a Mac OS X, `brew install graphviz`. For other platforms, see [_https://graphviz.org/download/_](https://graphviz.org/download/).)

In [3]:
import collections
import matplotlib.pyplot as plt

import utils

## Part 1: Assembling the TeleTubby Genome

For this part, you're given reads generated while trying to sequence the DNA of a mysterious unknown organism with a _tiny_ genome. Some of the course staff is not sure what a TeleTubby is, but I've been assured that no cute creatures were harmed in producing this data (which was generated synthetically).

By answering the following questions, you will learn how to assemble the original genome sequence from sequence reads.

Sequencing data is often stored in the FASTQ file format, with is a simple ASCII format that is somewhat human-readable. 

In _TeleTubby.fastq_ (which is included in the repository you forked), you will find the data that was read from the TeleTubby genome. Each four lines of the file repeat the same pattern. For example,
```
@TeleTubby Genome: Project 1
TAAAATGG
+
HAICDF5I
```
The first line contains the metadata that encodes the name of the read, the experiment type, the kind of sequencing machine used, etc. 

The second line is the sequence of bases that was read. 

The third line is just a placeholder.

The fourth line is a sequence of base qualities that encodes the qualities for the corresponding bases in the sequence line. We will discuss and use this for Problem 2.

Read in the data from `TeleTubby.fastq`:

In [88]:
# Read sequence reads (error-free) from file
sequence_reads, qualities = utils.read_fastq('TeleTubby.fastq')
# print(qualities)
# print(sequence_reads)

### Melting Temperature

The GC-content (or the ratio of G and C nucleotides) is related to the melting temperature of a DNA double helix structure. 

The following equation can be used to estimate the melting temperature (in degrees Celsius) of DNA for a particular sequence:

\begin{equation*}
t_m = 64.9+0.41(\%GC)-\frac{500}{\text{length of sequence}}
\end{equation*}

As a reference, the human genome is known to have between 35%-60% GC-content. 

<div class="alert alert-success">

**Problem 1.** Calculate the melting temperature for the TeleTubby genome using the formula above. Assume that the sequence is 200 nucleotides long, and that the provided read data has equal coverage everywhere so the G-C distribution in the reads is very close to the actual G-C distribution for the genome. 
    </div>

In [89]:
# Write code here for calculating the %GC content from the sequence_reads, and computing the estimated melting temperature.

# first let's calculate %GC for each snippet
gc = []
for sequence in sequence_reads:
    counter = 0
    size = len(sequence)
    for i in range(size):
        char = sequence[i]
        if char == 'G' or char == 'C':
            counter += 1
    gc.append((counter/size) * 100)

#print(gc)

# we need one number for the calculation, so let's average these together
average = 0;
for i in gc:
    average += i;
average = average / len(gc)

#print(average)

# cool, now let's just plug into the equation
t = 0.41 * average
t += 64.9
t -= (500/300)

# print(t)

In [90]:
# Print out temperature in Celsius
print("my estimation for this genome's melting temperature is " + str(t) + " degrees C.")

my estimation for this genome's melting temperature is 84.74783845278726 degrees C.


## Interpreting Read Quality

Phred33 quality scores are represented as the character with an ASCII code equal to its value + 33 (to make them easy to print alongside genome sequences). The Phred scores $Q$ are related to error probabilities $P$ as: $Q = -10 \log_{10}(P)$. The table linked here provides a mapping from the Phred values provided in a fastq file and the $Q$ values: [Quality Score Encoding](https://support.illumina.com/help/BaseSpace_OLH_009008/Content/Source/Informatics/BS/QualityScoreEncoding_swBS.htm) 

<div class="alert alert-success">
    
**Problem 2.** Using the Phred scores, identify the _least likely to be correct_ read in the TeleTubby.fastq data, which is the read with the lowest probability that it contains _no_ errors.

</div>

In [91]:
# Write your code for Problem 2 here. 
# It should output the read (nucleotide sequence) with the lowest probability of containing no errors.

# ignore this - i'm just messing around here

#test = ord('C') - 33
#print(test)
#test = test/-10
#test = 10**test
#print(test)

# anyway,
# for each read, we should convert the Q-scores to error probabilities,
# do math to figure out the total probability,
# then pick the read with the highest error

# convert Q-scores to error probs
errors = []
for read in qualities:
    read_errors = []
    for i in range(len(read)):
        Q = ord(read[i]) - 33
        Q = Q/-10
        Q = 10**Q
        read_errors.append(Q)
        
    # do math to figure out the total probability
    # the probability of there being an error in the sample is 1 minus the probability that there are no errors
    # and the probability that there are no errors can just be multiplied together (assuming mutually exclusive)
    count = 1
    for j in read_errors:
        no_error = 1 - j
        count = count * no_error
    errors.append(1 - count)

# print(errors)

# now we just want the read with the highest error

#highest = max(errors) <-- doesn't actually work, we want to return which read had the highest one
#print(highest)

highest = 0
the_one = 0
for k in range(len(errors)):
    if (errors[k] > highest):
        highest = errors[k]
        the_one = k
        
print("the read with the highest probability of containing an error is the "
      + str(the_one) + "th read (" + sequence_reads[the_one] + ") with probability " + str(highest)
     + " of having an error.")

the read with the highest probability of containing an error is the 14th read (GAGCCACC) with probability 0.04407109983095037 of having an error.


Note: For the following problems, you can ignore the read quality scores, and safely assume there are no actual errors in the provided TeleTubby reads.

## Frequency analysis

Looking at repetitions in the sequence can be helpful in estimating the "redudancy" in the organisms. Eucaryote genomes have a lot of redundancy, while smaller organisms like bacteria have highly packed genomes. One heuristic to estimate this before actually performing the assembly could be looking at how often certain $k$-mers are repeated.

<div class="alert alert-success">
    
**Problem 3.**
    Print out the 3 most frequent <em>k</em>-mers in the TeleTubby reads with their frequencies. (As in Problem 1, doing this from the reads only produces the same result as from the genome if the read coverage is equal everywhere, but it should be a close estimate.)
</div>

In [92]:
# Your code here

# these snippets are just 8 characters long
# so we can prolly just brute-force this without some fancy algorithm
# (though finding some better way to do this would be p cool im just not smart enough for that lmao)

mers = {} # data structure to hold our counts

for i in range(8):
    # we want to find all -mers of length 1, then 2, then 3, then 4...
    i_mers = {}
    i += 1 # range(8) starts at 0 and goes to 7, but we want it to start at 1 and go to 8

    for sequence in sequence_reads:
        for j in range(len(sequence)):
            if j+i <= len(sequence): # our data gets messed up if we don't check for this (trust me)
                the_mer = sequence[j:j+i] # grab a -mer of length i
                if the_mer in i_mers.keys():
                    i_mers[the_mer] += 1 # if we've already got it in the database then just increment it
                else:
                    i_mers[the_mer] = 1 # otherwise let's add it to the data structure

    mers[str(i)] = i_mers # save these counts to the bigger database

# print(mers['1']) <-- uncomment these if you wanna figure out wtf im doing up there
# print(mers['2'])
# print(mers['3'])

# now let's figure out what the most frequent 3 are

first = 0
first_mer = ''
second = 0
second_mer = ''
third = 0
third_mer = ''

for i in range(7):
    i += 2 # somehow i don't think we should consider "1-mers" when deciding the most frequent k-mers... let's skip them
    i_mers = mers[str(i)]
    for j in i_mers:
        if i_mers[j] > first:
            first = i_mers[j]
            first_mer = j
        elif i_mers[j] > second:
            second = i_mers[j]
            second_mer = j
        elif i_mers[j] > third:
            third = i_mers[j]
            third_mer = j
            
# print(first, first_mer)
# print(second, second_mer)
# print(third, third_mer)
print("The most frequent k-mer is " + first_mer + " (appears " + str(first) + " times)"
      + "\nThe second and third most frequent k-mers are " + second_mer + " (" + str(second) + " times) and "
      + third_mer + " (" + str(third) + " times)")

# this is still pretty unsatisfying tbh, 
# I could have told you that 2-mers would be the most common even without running this fancy program
# so let's also print some larger k-mers

# just copy-pasted from above
first = 0
first_mer = ''
second = 0
second_mer = ''
third = 0
third_mer = ''
for i in range(6):
    i += 3
    i_mers = mers[str(i)]
    for j in i_mers:
        if i_mers[j] > first:
            first = i_mers[j]
            first_mer = j
        elif i_mers[j] > second:
            second = i_mers[j]
            second_mer = j
        elif i_mers[j] > third:
            third = i_mers[j]
            third_mer = j
            
print("\nExcluding 2-mers, the most frequent k-mer is " + first_mer + " (appears " + str(first) + " times)"
      + "\nThe second and third most frequent k-mers are " + second_mer + " (" + str(second) + " times) and "
      + third_mer + " (" + str(third) + " times)")

# neat that TCG is the third most common, I love Trading Card Games :)

# print(mers['7'])

The most frequent k-mer is GA (appears 198 times)
The second and third most frequent k-mers are AT (174 times) and AG (132 times)

Excluding 2-mers, the most frequent k-mer is TGA (appears 90 times)
The second and third most frequent k-mers are ATG (54 times) and TCG (38 times)


## Greedy Assembly

Given a set of sequence fragments, the objective of assembly can be viewed as finding the shortest sequence that contains all the fragments.

One of the approaches to assemble the genome from the given reads is a greedy algorithm:

```
while (len(fragments) > 1):
   calculate pairwise alignments of all pairs of fragments
   merge the two fragments with the largest overlap
genome = fragment[0] # the single remaining fragment is the genome
```


<div class="alert alert-success">
    
**Problem 4 (a)**. What would the runtime be of this algorithm, given $n$ $k$-mer reads? (If you are not sure how to express your answer to this question, review [_Cost of Computation_](https://computingbiology.github.io/complexity/).)
    </div>

<i>Answer</i>: Calculating all pairwise allignments takes n^2, we do this n times, so n^3 runtime overall

<div class="alert alert-success">

**Problem 4 (b)**. Is this algorithm guaranteed to find the correct genome? (A good answer will define what _correct_ means here, and explain why the algorithm is or is not guaranteed to find it.)

<i>Answer</i>: Well, we could define "correct genome" to be the one that was actually in the organism before we chopped it up and analyzed it. If we choose to define it like that the answer is clearly no. Like, let's say we had these snippets to work with:

ATTA TAGA ACA AGA 
          
Applying our algorithm, TAGA and AGA overlap in three places, so we can combine those:

ATTA TAGA ACA

Then, ATTA and TAGA overlap in two places:

ATTAGA ACA

Finally, ATTAGA and ACA overlap in one place. So in the end, we get this:

ATTAGACA

Or do we get this?

ACATTAGA

Our algorithm doesn't really help us here. Only one of these genes is what was actually in the organism before it got all chopped up, but our algorithm can't really figure out which one is more "correct" because they're the same length.

And that's even assuming the full genome was this short to begin with - I started writing this section by cutting up this genome, actually:

ATTAGAACAAGA

So, no, if by "correct" we mean "what the genome was before we cut it up", this algorithm isn't guaranteed to give us that. But that's kind of boring, like, we said as much in class. Let's change "correct" to mean "the shortest sequence that contains all the snippets". Does this algorithm do that?

If I remember 4102 correctly, we can show that this algorithm always produces the correct genome by doing a proof by cases and comparing it to a hypothetical, perfectly optimized algorithm. Every time this greedy algorithm decides to combine two fragments, there are two cases:

<b>Case 1</b>: The optimal algorithm also combined those fragments the same way

In this case, our greedy algorithm exactly matches the optimal algorithm.

<b>Case 2</b>: The optimal algorithm did not combine those fragments that way

In this case, the optimal algorithm must have combined those two fragments some other way. From here, there are two more possiblilties:

<b>Case 2.1</b>: The optimal algorithm combined those fragments in a different way that led to the same amount of overlap

In this case, the answers given by the optimal algorithm and our greedy algorithm are equally correct, since they have the same length.

<b>Case 2.2</b>: The optimal algorithm combined those fragments in a different way that led to a different amount of overlap

Our greedy algorithm already identified the combination that led to the highest overlap, so if the optimal algorithm chose to overlap them differnetly, then it must have overlapped them in a way that led to less overlap and, thus, a higher overall length. This means the so-called "optimal" solution wasn't actually optimal at all, and our greedy algorithm is better.

So, in every case, our greedy algorithm is either just as correct as the optimal solution or more correct than that solution. Therefore, we can conclude that this greedy algorithm always produces the correct answer, so long as we choose to define "correct" this way.

## Graph-based Assembly

Graphs for genome assembly can be constructed in two ways:

- Overlap graph: Processing $k-$mers as nodes, with $(k-1)-$mers as edges, and
- de Bruijn graph: Processing $k-$mers as edges, with $(k-1)-$mers as nodes.

A de Bruijn graph can be processed to find Euler paths, while Overlap graphs can be processed to find Hamiltonian paths. Both of these methods can be used reconstruct the original genome.

<div class="alert alert-success">
    
**Problem 5.**  Use one of these two techniques to reconstruct the TeleTubby genome from the provided sequence reads. 
    
</div>
    
We have provided some template code below that may be helpful (but feel free to ignore this is you prefer).

In [143]:
# Read reads into graph

def build_graph(sequence_reads):
    
    # ok look i'm not really sure what this skeleton code is doing (what's a gvmagic?)
    # so uhhh let's just build this graph from scratch
    # we'll encode it like this:
    #   the graph will be a dictionary
    #   keys are nodes
    #   values are an array of edges (tuples)
    #   each edge (tuple) has two values - the value of the edge (the k-mer) and the other node it's connected to
    #   wait i lied that's so unnecessary nvm the edges will just be the node it's connected to
    #   we already know what the value of the edge is because it's just the overlap between the two
    
    graph = {}
    
    # going with overlap here
        
    # the nodes should be k-mers
    # so let's just grap all the samples since they all have the same length
    for sequence in sequence_reads:
        graph[sequence] = []
            
    # ok we've got our nodes, let's connect em' with edges
    # ...this is going to be expensive isn't it -_-
    # well it's at least n^2, so let's start there
    for i in graph.keys():
        for j in graph.keys():
            # we want to link these two if the last 7 of the first one match the first 7 of the second
            if i[1:] == j[0:-1]:
                # add an edge fron i to j
                graph[i].append(j)
    
        
    
    return graph

In [144]:
graph = build_graph(sequence_reads)

# print(graph)

In [145]:
# ok, now let's figure out this hamiltonian path
# i used this to help me figure this thing out:
# https://mathworld.wolfram.com/HamiltonianPath.html#:~:text=A%20Hamiltonian%20path%2C%20also%20called,cycle%20(or%20Hamiltonian%20cycle).

In [146]:
# Main assembly algorithm

# okay this is a mess lmao
# ill try to add a gratuitous amount of comments to this thing

def assemble_sequence(graph, path, already_visited, location): # recursive function, needs a lot of data passed down
    num = len(graph.keys())

    while(num != len(already_visited)): # don't stop until we visit every node
        
        nxt = graph[location] 
        viable = [] 
        
        # find all the 'viable' nodes i.e. all the ones we haven't visited before and that we can jump to
        for i in nxt: 
            if i not in already_visited:
                viable.append(i)
                
        if len(viable) == 0:
            # print("no hamiltonian path") # print statements for debugging :P
            return path
        
        # i don't think i had to do this part but i felt that if i could avoid using a recursive call, it's worth it
        # esp since glancing at the graph it looks like a lot of these nodes just have one outgoing edge
        elif len(viable) == 1:
            location = viable[0]
            path += location[7] # every new node we jump to adds one char to the path
            already_visited.append(location)
            # print(path) # more debugging - how do you think I noticed that CAGACCTG had no outgoing edges?
            
        # if there's more than one viable outgoing edge, we have to get ~funky~
        else:
            solutions = []
            for j in viable: # set up a recursive call for each viable path
                already_visited_local = already_visited # different calls want different lists of already_visited
                path_local = path # ditto for path
                location = j
                path_local += location[7]
                already_visited_local.append(location)
                try_this = assemble_sequence(graph, path_local, already_visited_local, location)
                solutions.append(try_this)
                    
            for k in solutions: # now that we've tried all the paths, pick the one that's the best and save it
                if len(k) > len(path):
                    path = k
        
    return path # ya done! 



In [147]:
# Output assembled sequence
# Hint: Sequence is 200 nucleotides long

# it doesn't matter where we start, since the cycle has to go through all the nodes
# so uhh let's start at 0
sequences = list(graph.keys())
path = sequences[0]
already_visited = [sequences[0]]
location = sequences[0]

# assmebled_seq = assemble_sequence(graph, path, already_visited, location)
# print(assmebled_seq, len(assmebled_seq), len(sequences))
# print(graph['CAGACCTG'])

# okay, here's the thing
# i misread some stuff up there
# we want a hamiltonian *path*, not a hamiltonian *cycle*
# why does this matter? CAGACCTG doesn't have any out going edges
# so eventually this recursive thing will get stuck at CAGACCTG and return that no hamiltonian path exists
# but a hamiltonian path *does* exist, just not a hamiltonian cycle

# my solution?
# ig we run it on every starting point and output the longest one
# oh my god this is SUCH a MESS >_>

possible_sequences = []
for i in sequences:
    path = i
    already_visited = [i]
    location = i
    possible_sequences.append(assemble_sequence(graph, path, already_visited, location))
    
da_best = ""
for i in possible_sequences:
    if len(i) > len(da_best):
        da_best = i

print(da_best, len(da_best))

# and sure enough, this one ends in CAGACCTG
# god this question was a headache LOL

CTCGAATCGTAAACGTGTGAATGTGTGATATGATGATTGATGAGTCCGATACGGGCGGATTCTGACGCAAAGCATCACCCCGTCGGGGTAAAAGAGCCTGACGACCGTGAGCATATGTCAGCATTTACCGGCGTTGCTCCCGACTTGTGCTCACCTGACAACATTAGGCAGGATAAGCTTATGAGATGACGCCCCCCCAGTGGTCAGATTTCCATCCCTGAGGATTTCGGTAGGACAATTTACCCATCGCGCGCCAGGGGTCGTAGGTAGAATTGCTCGGAGCCACCAGACCTG 294


<div class="alert alert-success"> 
    
**Problem 6.** Which of the two assembly methods did you use for problem 5, and why? (Hint: consider how the costs scale with the number of reads.)
    
</div>

## Part 2: Sequencing SARS-CoV-2 virus

Let's move on from TeleTubbies to real-world organisms. For this problem, you'll assemble a genome for a variant of the SARS-CoV-2 virus. You're given reads from <i>actual</i> genome sequencing runs in the provided `SARS-CoV2.fastq` file. The file is based on the first Covid genome that was submitted on 5 January 2020: [https://www.ncbi.nlm.nih.gov/nuccore/NC_045512](https://www.ncbi.nlm.nih.gov/nuccore/NC_045512) But, you should be reconstructing a sequence from the provided reads without looking at the source sequence.

<div class="alert alert-success">
    
**Problem 7.**  Reconstruct the SATS-CoV2 genome from the provided sequence reads using $k=25$. 
    
</div>

You can re-use your implementation from Problem 5 and simply run it on the new data (depending on what you did for Problem 5, that might be enough to solve this problem). 

Print out your reconstructed sequence to a file `output.txt` (and add this file to the repo you submit). For this part, we will still assume that all the reads are error-free. 

In [148]:
# Read sequence reads
sequence_reads_covid, qualities_covid = utils.read_fastq('SARS-CoV2.fastq')

In [149]:
# Read reads into graph
graph_covid = build_graph(sequence_reads_covid)
# print(graph_covid)

In [156]:
# Call main assembly algorithm
# assmebled_covid_seq = assemble_sequence(nodes_covid, edges_covid)

sequences_covid = list(graph_covid.keys())
num = len(sequences_covid)
counter = 0

# ok i literally can't run this on every possible reading
# it would just take too long
# don't believe me? of course you do literally just look at it lmao
# but if you actually don't believe me, run this
# every time it finishes calculating 1 (one) run of assemble_sequence, it'll spit out how many characters its solution was
# and how many more it has left to go
# i left this thing on for about an hour while i took a shower and did some cleaning up
# and it still wasn't finished
# so uhhh at this point i think i just have to accept that the solution i made is bad
# mission failed, we'll get em' next time i guess

possible_sequences_covid = []
for i in sequences_covid:
    counter += 1
    path = i
    already_visited = [i]
    location = i
    possible_sequences_covid.append(assemble_sequence(graph_covid, path, already_visited, location))
    print(len(possible_sequences_covid[-1]), "characters (" + str(counter) + "/" + str(num) + ")")
    
# if you were curious, i started a timer on my phone
# and determined that it took a minute and 23 seconds to complete 30 calls to assemble_sequence
# that's about 2.8 seconds per call
# which, at approx. 30,000 calls
# would take 83,000 seconds
# or just about 23 hours
# ...this solution really is terrible, huh
    
da_best_covid = ""
for i in possible_sequences_covid:
    if len(i) > len(da_best_covid):
        da_best_covid = i

print(da_best_covid, len(da_best_covid))

22923 characters (1/29871)
6305 characters (2/29871)
14814 characters (3/29871)
20995 characters (4/29871)
29599 characters (5/29871)
28619 characters (6/29871)
24708 characters (7/29871)
22807 characters (8/29871)
8700 characters (9/29871)
14657 characters (10/29871)
2272 characters (11/29871)
7967 characters (12/29871)
2037 characters (13/29871)
13952 characters (14/29871)
15953 characters (15/29871)
17503 characters (16/29871)
15905 characters (17/29871)
22382 characters (18/29871)
18573 characters (19/29871)
19675 characters (20/29871)
6718 characters (21/29871)
10767 characters (22/29871)
4229 characters (23/29871)
4348 characters (24/29871)
27549 characters (25/29871)
374 characters (26/29871)
20556 characters (27/29871)
13041 characters (28/29871)
25080 characters (29/29871)
24109 characters (30/29871)
2504 characters (31/29871)
16799 characters (32/29871)
14969 characters (33/29871)
13568 characters (34/29871)
3346 characters (35/29871)
29776 characters (36/29871)
8723 characte

KeyboardInterrupt: 

In [ ]:
# Write assembled sequence to file
with open("output.txt", "w") as f:
    f.write("solution runtime too slow to compute, see comments in notebook for details :(")

# Part 3: Error-Aware Assembly (Challenge Problem)

<div class="alert alert-warning">
    This problem is a "Challenge Problem". This means it is a problem of unknown difficulty that might be quite challenging (unlike the earlier problems, we don't have a reference solution for this one, or a clear idea how hard it might be). We do hope all students will at least attempt this and that more ambitious students will work hard to solve it and learn interesting things by the attempt (whether or not it is successful), but not get frustrated if you can't get to the desired answer.  As a "Challenge Problem" it means that you shouldn't be worried if you are not able to solve this, though, and you can get full expected credit on this assignment without answering it.
</div>


In the parts above, we assumed error-free reads while assembling $k$-mers. As much as we'd like that, actual reads can (and do) have errors, captured by their Phred scores. 

For this question, you're given raw, actual reads from sequencing runs. Download the reads from this file:
https://sra-pub-sars-cov2.s3.amazonaws.com/sra-src/SRR11528307/ABS2-LN-R1_cleaned_paired.fastq.gz.  


<div class="alert alert-success">
    
**Problem 8 (Challenge).** Give the reads in the linked fastq file above, including their Phred33 quality score, assemble the most likely genome. Your solution should output the assembled sequence in `challenge.txt`. Provide a brief explantion of how your algorithm works and interesting things you learned in developing it.
    
</div>
    
This is an open-ended question. You are free to use any approach to deal with the issue. Make sure you provide your code, along with any assumptions you may have.

_Write a description of your algorithm, and things you learned from working on this here._

In [ ]:
# implementation 

# so i've been going through a lot of personal stuff over the past couple weeks
# it's part of the reason why i'm working alone on this, actually
# i didn't think it'd be fair to try to work with someone else when i'm not sure if i'll be able to do my fair share

# anyway point is i do think that this is an interesting question
# and i'd like to attempt to answer it
# but i just don't have it in me rn

# still, my computer took like a billion years to finish calculating the covid genome (go figure)
# so i had time to think over this question
# and while i don't really have an answer to it
# i do have some thoughts
# i'll leave them here just to say i *did* think about it, and not much more


# the first part seems to be figuring out what "most likely genome" even means
# let's say that the most likely genome is the one that has the least probability of containing an error
# simple, right? but this is still sort of weird to me
# like, if we have two reads - ATTAGA and AGACTTA, what happens to the error probability when we overlap them on AGA?
# i guess we make it so the error becomes the probability that either read had an error, right?
# because they're both "included" in our final read
# but then this is a problem, right?
# because if we sequence this genome the same way we sequenced the COVID/TeleTubby one
# then we'll include every read that was collected, regardless of how much error it had
# so won't the error probability be equal among all the sequenced genes?
# that doesn't seem very helpful

# so clearly we need a different approach
# something that i think might have potential is using a "risk score" or something
# where Phred scores indicating high error probability increase the "risk score"
# but this increase can be offset by high overlap between reads
# so like if we have three reads - TGACCAAGTGAC, ACCAAGTGACCCG, and GACCAAGTGACC
# if the ACCAAGTGAC part might has a lot of errors in one of the reads, that'd increase the "risk score"
# but then the fact that we read it in three times indicates that it's probably part of the real genome
# so we would decrease the "risk score" the more overlap we can find with questionable data


# how could we implement that?
# i think that it's best if we leave calculating the "risk score" up to the biologists
# so let's let our sequencing function take in a couple functions as arguments that augment the risk score
# maybe the two of them looks like this:

def risk_score_increase(phread, risk):
    risk += phread
    return risk

def risk_score_decrease(overlap, risk):
    risk -= (overlap * 5)
    return risk

# and then whenever we find a way to combine two reads, we run both of these to modify the risk score
# and at the end we output the shortest genome that incorporates all the reads and has the lowest risk score

# or something like that, i dunno, that's all i got

   <div class="alert alert-block alert-danger">
    <center>
        
 **Remember to follow the submission directions above to submit your assignment by 4:59pm on Tuesday, 8 February.**
    
 </center>
 </div>